In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
------------------------------------------------------------------


In [3]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

fatal: destination path 'apex' already exists and is not an empty directory.
/content/gdrive/MyDrive/first_try_of_fastai/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-u99l94zf
Created temporary directory: /tmp/pip-req-tracker-lw68tcuy
Created requirements tracker '/tmp/pip-req-tracker-lw68tcuy'
Created temporary directory: /tmp/pip-install-e92t6jrm
Processing /content/gdrive/MyDrive/first_try_of_fastai/apex
  Created temporary directory: /tmp/pip-req-build-fqi2ry_2
  Added file:///content/gdrive/MyDrive/first_try_of_fastai/apex to build tracker '/tmp/pip-req-tracker-lw68tcuy'
    Running setup.py (path:/tmp/pip-req-build-fqi2ry_2/setup.py) egg_info for package from file:///content/gdrive/MyDrive/first_try_of_fastai/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-req-build-fqi2ry_2/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-fqi2ry_2/pip-egg-info/apex.egg-info/PKG-INFO
    

In [4]:
#export
import os
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai/exp')
from nb_11 import *
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

In [5]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)

In [6]:
#export
def cos_1cycle_anneal(start, high, end):
    return [sched_cos(start, high), sched_cos(high, end)]
    
def create_phases(phases):
    phases = listify(phases)
    return phases + [1-sum(phases)]
    
def noop(x): return x

class Flatten(nn.Module):
  def forward(self, x): return x.view(x.size(0), -1)

act_fn = nn.ReLU(inplace=True)

def init_cnn(m):
  if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
  if isinstance(m, (nn.Conv2d, nn.Linear)): nn.init.kaiming_normal_(m.weight)
  for l in m.children(): init_cnn(l)

def cnn_learner(arch, data, loss_func,opt_func, c_in=None, c_out=None,
                lr=1e-2, cuda=True, norm=None, progress=True, mixup=0,xtra_cb=None, **kwargs):
  cbfs = [partial(AvgStatsCallback,accuracy)] + listify(xtra_cb)
  if progress: cbfs.append(ProgressCallback)
  if cuda: cbfs.append(CudaCallback)
  if norm: cbfs.append(partial(BatchTransformXCallback, norm))
  if mixup: cbfs.append(partial(MixUp, mixup))
  arch_args = {}
  if not c_in: c_in = data.c_in
  if not c_out: c_out = data.c_out
  if c_in: arch_args['c_in'] = c_in
  if c_out: arch_args['c_out'] = c_out
  return Learner(arch(**arch_args), data, loss_func, opt_func=opt_func, lr=lr, cb_funcs=cbfs, **kwargs)

class CategoryProcessor(Processor):
    def __init__(self): self.vocab=None

    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            self.vocab = uniqueify(items)
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return self.otoi[item]

    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return self.vocab[idx]

def xresnet18 (**kwargs): return XResNet.create(1, [2,2,2 ,2], **kwargs)
def xresnet34 (**kwargs): return XResNet.create(1, [3,4,6 ,3], **kwargs)
def xresnet50 (**kwargs): return XResNet.create(4, [3,4,6 ,3], **kwargs)
def xresnet101(**kwargs): return XResNet.create(4, [3,4,23,3], **kwargs)
def xresnet152(**kwargs): return XResNet.create(4, [3,8,36,3], **kwargs)

class XResNet(nn.Sequential):
  @classmethod
  def create(cls, expansion, layers, c_in=3, c_out=1000):
    nfs = [c_in, (c_in+1)*8, 64, 64]
    stem = [conv_layer(nfs[i], nfs[i+1], stride=2 if i==8 else 1)
            for i in range(3)]
    
    nfs = [64//expansion, 64, 128, 256, 512]
    res_layers = [cls._make_layer(expansion, nfs[i], nfs[i+1],
                                 n_blocks=1, stride=1 if i==0 else 2)
                  for i,l in enumerate(layers)]

    res = cls(
        *stem,
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        *res_layers,
        nn.AdaptiveAvgPool2d(1), Flatten(),
        nn.Linear(nfs[-1]*expansion, c_out),
    )

    init_cnn(res)
    return res

  @staticmethod
  def _make_layer(expansion, ni, nf, n_blocks, stride):
      return nn.Sequential(
          *[ResBlock(expansion, ni if i== 0 else nf, nf, stride if i==0 else 1)
          for i in range(n_blocks)])

def conv(ni, nf, ks=3, stride=1, bias = False):
  return nn.Conv2d(ni, nf, kernel_size = ks, stride=stride, padding=ks//2, bias =bias)

def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, act=True):
  bn = nn.BatchNorm2d(nf)
  nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
  layers = [conv(ni, nf, ks, stride=stride), bn]
  if act: layers.append(act_fn)
  return nn.Sequential(*layers)

class ResBlock(nn.Module):
  def __init__(self, expansion, ni , nh, stride = 1):
    super().__init__()
    nf,ni = nh*expansion,ni*expansion
    layers =  [conv_layer(ni, nh, 1)]
    layers += [
        conv_layer(nh, nf, 3, stride=stride, zero_bn=True, act=False)
    ] if expansion==1 else [
        conv_layer(nh, nh, 3, stride=stride),
        conv_layer(nh, nf, 1, zero_bn = True, act=False)
    ]
    self.convs = nn.Sequential(*layers)
    self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
    self.pool = noop if stride==1 else nn.AvgPool2d(2)

  def forward(self, x): return act_fn(self.convs(x) + self.idconv(self.pool(x)))

class GeneralRelu(nn.Module):
    def __init__(self, leak=None, sub=None, maxv=None):
        super().__init__()
        self.leak,self.sub,self.maxv = leak,sub,maxv

In [7]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(128, scale=(0.35, 1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(128), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y = CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers = 8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [8]:
len(il)

12954

In [9]:
loss_func = LabelSmoothingCrossEntropy()
#arch = partial(xresnet18, c_out=10)
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [10]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [11]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end = 0.95):
  phases = create_phases(pct_start)
  sched_lr = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
  sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
  return [ParamScheduler('lr', sched_lr),
          ParamScheduler('mom', sched_mom)]

In [12]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [13]:
#learn.fit(40, cbsched)
learn.fit(4, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.178407,0.225374,2.263396,0.205905,00:50
1,2.099109,0.270028,2.320185,0.212013,00:50
2,1.929697,0.349252,1.914365,0.354798,00:50
3,1.733941,0.450859,1.701280,0.469076,00:50


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/content/gdrive/MyDrive/first_try_of_fastai/exp/nb_09.py:140: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  state['grad_avg'].mul_(mom).add_(state['mom_damp'], p.grad.data)


In [14]:
st = learn.model.state_dict()

In [15]:
type(st)

collections.OrderedDict

In [16]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs.0.1.running_var, 5.0.convs.0.1.num_batches_tracked, 5.0.convs.1.0.weight, 5.0.convs.1.1.weight, 5.0.convs.1.1.bias, 5.0.convs.1.1.running_mean, 5.0.convs.1.1.running_var, 5.0.convs.1.1.num_batches_tracked, 5.0.idconv.0.weight, 5.0.idconv.1.weight, 5.0.idconv.1.bias, 5.0.idconv.1.running_mean, 5.0.idconv.1.

In [17]:
st['10.bias']

tensor([-0.0057,  0.0008,  0.0112, -0.0004, -0.0008, -0.0063, -0.0064,  0.0029,
         0.0071,  0.0144], device='cuda:0')

In [18]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok = True)

In [19]:
torch.save(st, mdl_path/'iw5')

**PETS**

In [20]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [21]:
pets.ls()

[PosixPath('/root/.fastai/data/oxford-iiit-pet/images'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations')]

In [22]:
pets_path = pets/'images'

In [23]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [24]:
il

ImageList (7390 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/yorkshire_terrier_11.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Sphynx_146.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/boxer_187.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/pomeranian_128.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/samoyed_128.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Abyssinian_196.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Russian_Blue_128.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Bombay_98.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Maine_Coon_43.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Sphynx_17.jpg'), ...]
Path: /root/.fastai/data/oxford-iiit-pet/images

In [25]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [26]:
random.seed(42)

In [27]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [28]:
sd

SplitData
Train: ImageList (6667 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/yorkshire_terrier_11.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/boxer_187.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/pomeranian_128.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/samoyed_128.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Abyssinian_196.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Russian_Blue_128.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Maine_Coon_43.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_31.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Egyptian_Mau_185.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/leonberger_184.jpg'), ...]
Path: /root/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Sphynx_146.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/ima

In [29]:
n = il.items[0].name; n

'yorkshire_terrier_11.jpg'

In [30]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'yorkshire_terrier'

In [31]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [32]:
proc = CategoryProcessor()

In [33]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [34]:
', '.join(proc.vocab)

'yorkshire_terrier, boxer, pomeranian, samoyed, Abyssinian, Russian_Blue, Maine_Coon, scottish_terrier, Egyptian_Mau, leonberger, staffordshire_bull_terrier, Bengal, german_shorthaired, american_pit_bull_terrier, basset_hound, Bombay, chihuahua, newfoundland, pug, British_Shorthair, Ragdoll, havanese, Siamese, english_cocker_spaniel, Birman, shiba_inu, Persian, wheaten_terrier, miniature_pinscher, japanese_chin, beagle, Sphynx, keeshond, english_setter, great_pyrenees, saint_bernard, american_bulldog'

In [35]:
ll.valid.x.tfms = val_tfms

In [36]:
c_out = len(proc.vocab)

In [37]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [38]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [ ]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.478012,0.090295,3.423317,0.095436,00:39


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


**CUSTOM HEAD**

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [ ]:
st = torch.load(mdl_path/'iw5')

In [ ]:
m = learn.model

In [ ]:
m.load_state_dict(st)

In [ ]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [ ]:
xb, yb = get_batch(data.valid_dl, learn)

In [ ]:
# added this functionality to send the model to cuda, was causing errors when model was on gpu
m_cut = m_cut.cuda()

In [ ]:
pred = m_cut(xb)

In [ ]:
pred.shape

In [ ]:
ni = pred.shape[1]

In [ ]:
#export
class AdaptiveConcatPool2d(nn.Module):
  def __init__(self, sz=1):
    super().__init__()
    self.output_size = sz
    self.ap = nn.AdaptiveAvgPool2d(sz)
    self.mp = nn.AdaptiveMaxPool2d(sz)
  def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [ ]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out)
)

In [ ]:
learn.model = m_new

In [ ]:
learn.fit(5, cbsched)

**adapt model**

In [ ]:
def adapt_model(learn, data):
  cut = next(i for i,o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
  m_cut = m[:cut]
  xb, yb = get_batch(data.valid_dl, learn)
  # added this functionality to send the model to cuda, was causing errors when model was on gpu
  m_cut = m_cut.cuda()
  pred = m_cut(xb)
  ni = pred.shape[1]
  m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out))
  learn.model = m_new

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

In [ ]:
adapt_model(learn, data)

In [ ]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [ ]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

In [ ]:
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [ ]:
learn.fit(5, cbsched, reset_opt=True)

**BATCH NORM TRANSFER** 

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

In [ ]:
def apply_mod(m, f):
  f(m)
  for l in m.children(): apply_mod(l, f)

def set_grad(m , b):
  if isinstance(m, (nn.Linear, nn.BatchNorm2d)): return
  if hasattr(m , 'weight'):
    for p in m.parameters(): p.requires_grad_(b)

In [ ]:
apply_mod(learn.model, partial(set_grad, b=False))

In [ ]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

In [ ]:
apply_mod(learn.model, partial(set_grad, b=True))

In [ ]:
learn.fit(5, cbsched, reset_opt=True)

In [ ]:
learn.model.apply(partial(set_grad, b = False))

**DISCRIMINATIVE LR AND PARAM GROUPS**

In [ ]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [ ]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

In [ ]:
def bn_splitter(m):
  def _bn_splitter(l, g1, g2):
    if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
    elif hasattr(l, 'weight'): g1 += l.parameters()
    for ll in l.children(): _bn_splitter(ll, g1, g2)

  g1,g2 = [],[]
  _bn_splitter(m[0], g1, g2)

  g2 += m[1:].parameters()
  return g1,g2

In [ ]:
a,b = bn_splitter(learn.model)

In [ ]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [ ]:
Learner.ALL_CBS

In [ ]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [ ]:
cb_types.after_backward

In [ ]:
#export
class DebugCallback(Callback):
  _order = 999
  def __init__(self, cb_name, f=None): self.cb_name, self.f = cb_name, f
  def __call__(self, cn_name):
    if cb_name == self.cb_name:
      if self.f: self.f(self.run)
      else: set_trace()

In [ ]:
#export 
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0,95):
  phases = create_phases(pct_start)
  sched_lr = [combine_sched(phases, cos_1cycle_anneal(lr.10., lr, lr/1e5))
              for lr in lrs]
  sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
  return [ParamScheduler('lr', sched_lr),
          Paramscheduler('mom', sched_mom)]

In [ ]:
dsci_lr_sched = sched_1cycle([0, 3e-2], 0.5)

In [ ]:
learn = cnn_learner(xresnet18, data loss_func, opt_func,
                    c_out = 10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

In [ ]:
def _print_det(o):
  print(len(o.opt.param_groups), o.opt.hypers)
  raise CancelTrainException()

learn.fit(1, disc_sched + [DebugCallback(cb_types.after_batch, _print_det)])

In [ ]:
learn.fit(3, disc_lr_sched)

In [ ]:
disc_lr_sched = sched_1cycle([1e-3, 1e-2], 0.3)

In [ ]:
learn.fit(5, disc_lr_sched)

**EXPORT**

In [ ]:
!pip install fire

In [ ]:
!python notebook2script.py 11a_train_imagenette